In [1]:
# Import Dependencies
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import html5lib
import lxml
from splinter import Browser
import pymongo
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# Set the executable path and initialize Splinter

executable_path = {"executable_path": "./chromedriver.exe"}
browser = Browser("chrome", **executable_path)

In [3]:
# NASA Mars News 
# Scrape the [Mars News Site](https://redplanetscience.com/) and collect the latest News Title and Paragraph Text. 
# Assign the text to variables that you can reference later.

# Visit the Maars News Site provided in the insructions

url = 'https://redplanetscience.com/'
browser.visit(url)

In [4]:
# News Title

# Using Beautiful Soup, the broswer html will be converted to a soup object.Convert the browser html to a soup object.
# To collect the latest News Title and Paragraph Text. I am going to the site and inspect the onject to get the classes for
# the title and paragraph. According to inspect process <div class="content_title> is the class
# Note: Can use soup because that is the name assigned when importing in bs.

html = browser.html
soup = bs(html, 'html.parser')
news_title = soup.find('div', class_='content_title')
news_title.text


'NASA Adds Return Sample Scientists to Mars 2020 Leadership Team'

In [5]:
# After locating the headline, referenced the news_title variable and the .text.strip() functions to make the news title
# regular text without the html markup
#https://python.hotexamples.com/examples/bs4/BeautifulSoup/strip/python-beautifulsoup-strip-method-examples.html

news_title = news_title.text.strip()
print(news_title)

NASA Adds Return Sample Scientists to Mars 2020 Leadership Team


In [6]:
#News Paragraph 

# According to the inspect process <div class="article_teaser_body">

news_paragraph = soup.find('div', class_='article_teaser_body')
news_paragraph


<div class="article_teaser_body">The leadership council for Mars 2020 science added two new members who represent the interests of scientists destined to handle and study the first samples from Red Planet.</div>

In [7]:
# After locating the paragraph, referenced the news_paragraph variable and the .text.strip() functions to make the 
# paragraph regular text without the html markup
#https://python.hotexamples.com/examples/bs4/BeautifulSoup/strip/python-beautifulsoup-strip-method-examples.html

news_paragraph = news_paragraph.text.strip()
print(news_paragraph)

The leadership council for Mars 2020 science added two new members who represent the interests of scientists destined to handle and study the first samples from Red Planet.


In [8]:
# JPL Mars Space Images - Featured Image

In [9]:
# Visit the url for the Featured Space Image site [here](https://spaceimages-mars.com).
# Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the 
# url string to a variable called `featured_image_url`.
# Make sure to find the image url to the full size `.jpg` image.
# Make sure to save a complete url string for this image.

url_2 = "https://spaceimages-mars.com/"
browser.visit(url_2)


html = browser.html
soup = bs(html, 'html.parser')
img = soup.find('img', class_='headerimage fade-in')
img_link = img['src']

featured_image_url = url_2 + img_link


In [10]:
###Scrape the table containing facts about diameter of Mars
#!pip install html5lib

url_3 = "https://galaxyfacts-mars.com/"

In [11]:
df = pd.read_html(url_3)
display(df[1])
mars_df = (df[1])
mars_df

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 ( Phobos & Deimos )
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 ( Phobos & Deimos )
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [12]:
# Converting df to html in case it is needed for the flask portion of the exercise
mars_html = mars_df.to_html(buf=None, columns=None, col_space=None, header=True, index=True, na_rep='NaN', formatters=None, float_format=None, sparsify=None,index_names=True, justify=None, bold_rows=True, classes=None, escape=True, max_rows=None, max_cols=None, show_dimensions=False, notebook=False)

In [13]:
mars_html

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>0</th>\n      <th>1</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 ( Phobos &amp; Deimos )</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2nd millennium BC</td>\n   

In [14]:
#Create empty dictionary for results
hemisphere_image_urls = {}

#Define the url
url_4 = "https://marshemispheres.com/"

#Go to main website
browser.visit(url_4)

#Read page as HTML
html = browser.html

#Parse HTML
soup = bs(html, 'html.parser')

#Find area that represents the list with hemispheres
nav_list = soup.find('div', class_="collapsible results")

#From the above area find every item in the list
all_hemispheres = nav_list.find_all('div', class_="item")

#Extract image link for each hemisphere
for hemisphere in all_hemispheres:
    #Find hemisphere name
    hemisphere_name = hemisphere.find('h3').text
    
    #Find the link to hemiphere page
    link = hemisphere.find('a', class_ = "itemLink product-item")['href']
    
    #Create a full URL for hemishphere page
    hemisphere_link = url_4 + link
    
    #Go to hemisphere page
    browser.visit(hemisphere_link)
    
    #Read page as HTML
    html = browser.html
    
    #Parse HTML
    soup = bs(html, 'html.parser')
    
    #Extract link to full image
    img = soup.find('img', class_='wide-image')
    hemisphere_img_link = img['src']

    #Create a full URL for hemisphere image
    hemisphere_image_url = url_4 + hemisphere_img_link
    
    #Add hemisphere name and hemisphere url to dictionary with results
    hemisphere_image_urls[hemisphere_name] = hemisphere_image_url

In [15]:
hemisphere_image_urls

{'Cerberus Hemisphere Enhanced': 'https://marshemispheres.com/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg',
 'Schiaparelli Hemisphere Enhanced': 'https://marshemispheres.com/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg',
 'Syrtis Major Hemisphere Enhanced': 'https://marshemispheres.com/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg',
 'Valles Marineris Hemisphere Enhanced': 'https://marshemispheres.com/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}